# **安装依赖库**

## **作者简介**

#### 作者：**⚡小闪电⚡**

#### B站主页
- [小闪电的B站主页](https://space.bilibili.com/122699831?spm_id_from=333.1007.0.0)

#### 交流群
欢迎加入AIDD交流群！  
加我微信（微信号: `xxxFLASHxxx`），邀请你进群。

## **安装依赖库**

In [ ]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install rdkit
!pip install networkx pandas
%matplotlib inline
import matplotlib.pyplot as plt
from torch_geometric.datasets import KarateClub
dataset = KarateClub()
print(f'Dataset: {dataset}:')
!pip install tabulate

# **第1部分：先试试GNN**

## **数据可视化**

In [ ]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
from transformers import AutoTokenizer, AutoModel
# from tqdm import tqdm # 演示数据不需要
from mpl_toolkits.mplot3d import Axes3D  # 必须导入用于3D绘图

# 设置绘图风格
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.dpi'] = 120
# 确保图表中不使用中文字体，防止乱码 (虽然本例要求全英文，但这是好习惯)
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['axes.unicode_minus'] = False

# ==========================================
# 1. 核心工具函数 (保持不变)
# ==========================================

def build_window_matrix(seq, window=1):
    """构建滑动窗口的邻接矩阵 (Baseline)"""
    L = len(seq)
    matrix = np.zeros((L, L))
    for i in range(L):
        start = max(0, i - window)
        end = min(L, i + window + 1)
        for j in range(start, end):
            if i != j:
                matrix[i, j] = 1
    return matrix

# ESM 模型加载
ESM_MODEL_NAME = "facebook/esm2_t6_8M_UR50D"
print(f"⏳ Loading ESM-2 Model ({ESM_MODEL_NAME})...") # 中文注释：控制台输出改为英文以防万一，注释保留中文
try:
    tokenizer = AutoTokenizer.from_pretrained(ESM_MODEL_NAME)
    esm_model = AutoModel.from_pretrained(ESM_MODEL_NAME, attn_implementation="eager").eval()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    esm_model = esm_model.to(device)
    print("✅ Model loaded successfully!")
except Exception as e:
    print(f"⚠️ Model load failed: {e}")
    exit()

def predict_esm_matrix(seq, k_neighbors=10):
    """ESM + Top-K + Backbone 混合建图"""
    inputs = tokenizer(seq, return_tensors="pt", truncation=True, max_length=1022).to(device)
    with torch.no_grad():
        outputs = esm_model(**inputs, output_attentions=True)

    # 1. 获取 Attention Map (取最后一层，平均所有头)
    attentions = outputs.attentions[-1].squeeze(0).mean(dim=0).cpu()
    # 去掉特殊 token <cls>, <eos>
    raw_matrix = attentions[1:-1, 1:-1].numpy()
    L = raw_matrix.shape[0]

    # 2. 对称化 & 去自环 (用于 Raw 展示和后续计算)
    raw_matrix_sym = (raw_matrix + raw_matrix.T) / 2
    np.fill_diagonal(raw_matrix_sym, 0)

    # 3. Top-K 二值化
    adj_matrix = np.zeros_like(raw_matrix_sym)
    k = min(k_neighbors, L - 1)
    # 对每一行找最大的K个索引
    topk_indices = np.argsort(raw_matrix_sym, axis=1)[:, -k:]
    for i in range(L):
        adj_matrix[i, topk_indices[i]] = 1
    # 再次确保二值图对称
    adj_matrix = ((adj_matrix + adj_matrix.T) > 0).astype(float)

    # 4. 注入 Backbone (确保不断裂)
    backbone = build_window_matrix(seq, window=1)
    final_adj_matrix = ((adj_matrix + backbone) > 0).astype(float)

    # 中文注释：关键修改，同时返回对称化后的原始Map和最终二值Map
    return raw_matrix_sym, final_adj_matrix

# ==========================================
# 2. 高级图结构可视化函数 (2D & 3D)
# ==========================================

def plot_graph_structure(adj_matrix, title, ax_2d, ax_3d, seq_len):
    """
    将邻接矩阵转换为 Graph 并绘制 2D 和 3D 结构图 (全英文标签)
    """
    # 1. 转换为 NetworkX 图
    rows, cols = np.where(adj_matrix > 0)
    edges = zip(rows.tolist(), cols.tolist())
    G = nx.Graph()
    G.add_nodes_from(range(seq_len))
    G.add_edges_from(edges)

    # 设置颜色映射 (根据序列位置 N端->C端 渐变)
    # 使用 spectral 或 viridis 模拟彩虹色，冷色调为N端，暖色调为C端
    node_colors = list(range(seq_len))
    cmap = plt.cm.Spectral

    # --- 2D 绘图 ---
    # 使用 Kamada-Kawai 布局，它比 spring_layout 更能体现化学分子的几何结构
    try:
        pos_2d = nx.kamada_kawai_layout(G)
    except:
        pos_2d = nx.spring_layout(G, k=0.15, iterations=20)

    nx.draw_networkx_nodes(G, pos_2d, ax=ax_2d, node_size=50,
                           node_color=node_colors, cmap=cmap, alpha=0.9)
    nx.draw_networkx_edges(G, pos_2d, ax=ax_2d, edge_color='gray', alpha=0.4, width=0.8)
    # 中文注释：确保标题是英文
    ax_2d.set_title(f"{title}\n(2D Projection)", fontsize=11, fontweight='bold')
    ax_2d.axis('off')

    # --- 3D 绘图 ---
    # 使用 3D Spring Layout
    pos_3d = nx.spring_layout(G, dim=3, k=0.5, iterations=50, seed=42)

    # 提取坐标
    xs = [pos_3d[i][0] for i in range(seq_len)]
    ys = [pos_3d[i][1] for i in range(seq_len)]
    zs = [pos_3d[i][2] for i in range(seq_len)]

    # 绘制节点
    ax_3d.scatter(xs, ys, zs, c=node_colors, cmap=cmap, s=40, depthshade=True)

    # 绘制边 (需要手动循环画线)
    for u, v in G.edges():
        ax_3d.plot([xs[u], xs[v]], [ys[u], ys[v]], [zs[u], zs[v]],
                   color='gray', alpha=0.3, linewidth=0.5)

    # 中文注释：确保标题是英文
    ax_3d.set_title(f"{title}\n(3D Force-Directed)", fontsize=11, fontweight='bold')
    # 隐藏坐标轴刻度，只保留空间感
    ax_3d.set_xticklabels([])
    ax_3d.set_yticklabels([])
    ax_3d.set_zticklabels([])
    ax_3d.grid(False)

# ==========================================
# 3. 主程序与可视化执行
# ==========================================

# 准备数据 (伪数据)
data = {'sequence': ['MLFSFFRNLCRVLYRVRVTGDTQALKGERVLITPNHVSFIDGILLGLFLPVRPVFAVYTSISQQWYMRWLKSFIDFVPLDPTQPMAIKHLVRLVEQGRPVVIFPEGRITTTGSLMKIYDGAGFVAAKSGATVIPVRIEGAELTHFSRLKGLVKRRLFPQITLHILPPTQVAMPDAPRARDRRKIAGEMLHQIMMEARMAVRPRETLYESLLSAMYRFGAGKKCVEDVNFTPDSYRKLLTKTLFVGRILEKYSVEGERIGLMLPNAGISAAVIFGAIARRRMPAMMNYTAGVKGLTSAITAAEIKTIFTSRQFLDKGKLWHLPEQLTQVRWVYLEDLKADVTTADKVWIFAHLLMPRLAQVKQQPEEEALILFTSGSEGHPKGVVHSHKSILANVEQIKTIADFTTNDRFMSALPLFHSFGLTVGLFTPLLTGAEVFLYPSPLHYR'*1]} # 稍长一点以便观察折叠
train_df = pd.DataFrame(data)

# 取一个样本
sample_seq = train_df.iloc[0]['sequence']
viz_len = min(len(sample_seq), 80) # 取前80个残基可视化，太长会很乱
viz_seq = sample_seq[:viz_len]

print(f"🔍 Analyzing sequence (length {viz_len})...")

# --- 计算 ---
# 1. Baseline 矩阵
mat_window = build_window_matrix(viz_seq, window=2)

# 2. ESM 矩阵 (中文注释：关键修改，接收两个返回值：原始Map 和 最终二值Map)
# k_neighbors=4 表示每个残基只看最强的4个远程关联
mat_raw_esm, mat_esm_final = predict_esm_matrix(viz_seq, k_neighbors=4)


# ==========================================
# 4. 绘图：分为两个 Figure 避免拥挤
# ==========================================

# --- Figure 1: 矩阵热力图对比 (中文注释：修改为3个子图以包含 Raw Map) ---
fig1, axes1 = plt.subplots(1, 3, figsize=(18, 5))

# Subplot 1: Baseline
sns.heatmap(mat_window, cmap="Greys", cbar=False, ax=axes1[0], square=True, vmin=0, vmax=1)
axes1[0].set_title("1. Baseline: Window Graph\n(Local Only)", fontweight='bold')
axes1[0].set_ylabel("Residue Index")

# Subplot 2: ESM Raw Attention (中文注释：新增的原始热力图)
# 使用 viridis 颜色图来展示连续的注意力强度值
sns.heatmap(mat_raw_esm, cmap="viridis", cbar=True, ax=axes1[1], square=True,
            cbar_kws={'label': 'Attention Intensity', 'shrink': 0.8})
axes1[1].set_title("2. ESM Raw Attention\n(Dense & Noisy)", fontweight='bold')
axes1[1].set_yticks([]) # 移除Y轴标签保持整洁

# Subplot 3: ESM Final Binary Matrix
sns.heatmap(mat_esm_final, cmap="Blues", cbar=False, ax=axes1[2], square=True, vmin=0, vmax=1)
axes1[2].set_title("3. ESM Final Graph\n(Sparse: Top-K + Backbone)", fontweight='bold')
axes1[2].set_yticks([])

# 中文注释：确保总标题是英文
plt.suptitle(f"Contact Map Processing Flow (Seq Len: {viz_len})", fontsize=14, y=0.98)
plt.tight_layout()
plt.show()


# --- Figure 2: 图结构可视化 (2D & 3D) ---
# 创建 2x2 的网格：上面是 Window Graph，下面是 ESM Graph
fig2 = plt.figure(figsize=(16, 12))

# 布局定义
# Row 1: Window Graph (2D, 3D)
ax_win_2d = fig2.add_subplot(2, 2, 1)
ax_win_3d = fig2.add_subplot(2, 2, 2, projection='3d')

# Row 2: ESM Graph (2D, 3D)
ax_esm_2d = fig2.add_subplot(2, 2, 3)
ax_esm_3d = fig2.add_subplot(2, 2, 4, projection='3d')

print("🎨 Rendering 2D/3D graph structures...")

# 绘制 Window Graph (使用英文标题)
plot_graph_structure(mat_window, "Baseline (Window-2)", ax_win_2d, ax_win_3d, viz_len)

# 绘制 ESM Graph (使用最终的二值矩阵，使用英文标题)
plot_graph_structure(mat_esm_final, "ESM-2 (Predicted Fold)", ax_esm_2d, ax_esm_3d, viz_len)

# 添加颜色条说明 (中文注释：确保标签是英文)
sm = plt.cm.ScalarMappable(cmap=plt.cm.Spectral, norm=plt.Normalize(vmin=0, vmax=viz_len))
sm.set_array([])
cbar_ax = fig2.add_axes([0.92, 0.15, 0.02, 0.7]) #右侧颜色条
cbar = fig2.colorbar(sm, cax=cbar_ax)
cbar.set_label('Residue Position (N-term -> C-term)')

# 中文注释：确保总标题是英文
plt.suptitle(f"Graph Structure Visualization (Sequence Length: {viz_len})", fontsize=16, y=0.95)
plt.subplots_adjust(left=0.05, right=0.9, wspace=0.1, hspace=0.2)
plt.show()

## **对比训练**

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.metrics import r2_score, mean_squared_error # 🔥 新增：用于计算R2和MSE

# ==========================================
# 1. 环境配置与模型加载
# ==========================================
# 设置绘图风格
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.dpi'] = 120
plt.rcParams['font.sans-serif'] = ['Arial'] # 防止中文乱码(如果有)
plt.rcParams['axes.unicode_minus'] = False

# 检查 GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# 加载 ESM-2 模型
ESM_MODEL_NAME = "facebook/esm2_t6_8M_UR50D"
print(f"⏳ Loading ESM Model: {ESM_MODEL_NAME}...")
try:
    tokenizer = AutoTokenizer.from_pretrained(ESM_MODEL_NAME)
    # 🔥 关键修复：添加 attn_implementation="eager" 以确保能返回 attention weights
    esm_model = AutoModel.from_pretrained(
        ESM_MODEL_NAME,
        attn_implementation="eager"
    ).eval().to(device)
    print("✅ ESM Model loaded successfully (with Eager Attention)!")
except Exception as e:
    print(f"⚠️ ESM Model load failed: {e}")
    exit()

# ==========================================
# 2. 图构建与特征提取函数
# ==========================================

def build_window_matrix(seq, window=2):
    """Baseline: 滑动窗口邻接矩阵"""
    L = len(seq)
    matrix = np.eye(L) # 初始化带自环
    for i in range(L):
        start = max(0, i - window)
        end = min(L, i + window + 1)
        for j in range(start, end):
            if i != j:
                matrix[i, j] = 1
    return matrix

def predict_esm_matrix(seq, k_neighbors=5):
    """Advanced: ESM Attention 接触图"""
    # 截断以防 OOM
    seq = seq[:1020]
    inputs = tokenizer(seq, return_tensors="pt", truncation=True, max_length=1022).to(device)

    with torch.no_grad():
        outputs = esm_model(**inputs, output_attentions=True)

    # 获取最后一层 Attention，平均所有头
    attentions = outputs.attentions[-1].squeeze(0).mean(dim=0).cpu()
    raw_matrix = attentions[1:-1, 1:-1].numpy()
    L = raw_matrix.shape[0]

    # 对称化
    raw_matrix = (raw_matrix + raw_matrix.T) / 2
    np.fill_diagonal(raw_matrix, 0)

    # Top-K 稀疏化
    adj_matrix = np.zeros_like(raw_matrix)
    k = min(k_neighbors, L - 1)
    if k > 0:
        topk_indices = np.argsort(raw_matrix, axis=1)[:, -k:]
        for i in range(L):
            adj_matrix[i, topk_indices[i]] = 1

    # 保证对称 + 融合 Backbone + 自环
    adj_matrix = ((adj_matrix + adj_matrix.T) > 0).astype(float)
    backbone = build_window_matrix(seq, window=1)
    final_adj = ((adj_matrix + backbone) > 0).astype(float)
    np.fill_diagonal(final_adj, 1.0)

    return final_adj

AA_VOCAB = "ACDEFGHIKLMNPQRSTVWY"
aa_to_int = {aa: i for i, aa in enumerate(AA_VOCAB)}

def sequence_to_node_features(seq):
    """One-Hot 编码"""
    seq = seq[:1020]
    L = len(seq)
    features = torch.zeros(L, 20)
    for i, aa in enumerate(seq):
        if aa in aa_to_int:
            features[i, aa_to_int[aa]] = 1.0
    return features

# ==========================================
# 3. 数据集构建 (训练集 + 测试集)
# ==========================================

class ProteinGraphDataset(torch.utils.data.Dataset):
    def __init__(self, df, graph_type='window', split_name='train'):
        self.data_list = []
        print(f"📦 Building {split_name} Dataset ({graph_type})...")

        # 遍历数据
        for idx, row in tqdm(df.iterrows(), total=len(df)):
            seq = row['sequence']
            try:
                label = float(row['solubility'])
            except:
                continue

            # A. 特征
            x = sequence_to_node_features(seq)

            # B. 邻接矩阵
            if graph_type == 'window':
                adj = build_window_matrix(seq, window=2)
            else:
                adj = predict_esm_matrix(seq, k_neighbors=5)

            # C. 转 Edge Index
            src, dst = np.nonzero(adj)
            edge_index = torch.tensor([src, dst], dtype=torch.long)
            y = torch.tensor([[label]], dtype=torch.float)

            self.data_list.append(Data(x=x, edge_index=edge_index, y=y))

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
        return self.data_list[idx]

# 读取 CSV
if os.path.exists('eSol_train.csv') and os.path.exists('eSol_test.csv'):
    train_df = pd.read_csv('eSol_train.csv')
    test_df = pd.read_csv('eSol_test.csv')
else:
    print("⚠️ 未找到数据集文件，生成伪数据用于演示...")
    fake_data = {'sequence': ['MKTVRQERLKSIVRILERSKEPVSGAQLAEELSVSRQVIVQDIAYLRSLGYNIVATPRGYVLAGG']*20, 'solubility': np.random.rand(20)}
    train_df = pd.DataFrame(fake_data)
    test_df = pd.DataFrame(fake_data)

# 限制数据量以便演示 (实际运行时请注释掉这两行)
# train_df = train_df.iloc[:50]
# test_df = test_df.iloc[:20]

print(f"数据量: Train={len(train_df)}, Test={len(test_df)}")

# 构建 4 个数据集对象
print("\n--- 构建 Baseline (Window) ---")
train_dataset_win = ProteinGraphDataset(train_df, 'window', 'Train')
test_dataset_win = ProteinGraphDataset(test_df, 'window', 'Test')

print("\n--- 构建 Advanced (ESM) ---")
train_dataset_esm = ProteinGraphDataset(train_df, 'esm', 'Train')
test_dataset_esm = ProteinGraphDataset(test_df, 'esm', 'Test')

# DataLoader
BATCH_SIZE = 32
train_loader_win = DataLoader(train_dataset_win, batch_size=BATCH_SIZE, shuffle=True)
test_loader_win = DataLoader(test_dataset_win, batch_size=BATCH_SIZE, shuffle=False)

train_loader_esm = DataLoader(train_dataset_esm, batch_size=BATCH_SIZE, shuffle=True)
test_loader_esm = DataLoader(test_dataset_esm, batch_size=BATCH_SIZE, shuffle=False)

# ==========================================
# 4. GCN 模型定义
# ==========================================

class GCNPredictor(torch.nn.Module):
    def __init__(self, num_node_features=20, hidden_dim=64):
        super(GCNPredictor, self).__init__()
        self.conv1 = GCNConv(num_node_features, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.conv3 = GCNConv(hidden_dim, 32)
        self.fc = nn.Linear(32, 1)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.2, training=self.training)

        x = F.relu(self.conv2(x, edge_index))
        x = F.dropout(x, p=0.2, training=self.training)

        x = F.relu(self.conv3(x, edge_index))

        x = global_mean_pool(x, batch)
        return self.fc(x)

# ==========================================
# 5. 训练与评估函数
# ==========================================

def evaluate(model, loader):
    model.eval()
    criterion = nn.MSELoss()
    total_loss = 0
    with torch.no_grad():
        for batch in loader:
            batch = batch.to(device)
            out = model(batch)
            loss = criterion(out, batch.y)
            total_loss += loss.item() * batch.num_graphs
    return total_loss / len(loader.dataset)

def train_experiment(train_loader, test_loader, model_name="Model", epochs=30):
    model = GCNPredictor().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.MSELoss()

    history = {'train_loss': [], 'test_loss': []}

    print(f"🚀 Training {model_name}...")
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for batch in train_loader:
            batch = batch.to(device)
            optimizer.zero_grad()
            out = model(batch)
            loss = criterion(out, batch.y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * batch.num_graphs

        avg_train = total_loss / len(train_loader.dataset)
        avg_test = evaluate(model, test_loader)

        history['train_loss'].append(avg_train)
        history['test_loss'].append(avg_test)

        if (epoch+1) % 5 == 0:
            print(f"   Epoch {epoch+1}/{epochs} | Train: {avg_train:.4f} | Test: {avg_test:.4f}")

    return model, history

# ==========================================
# 6. 执行训练
# ==========================================

# A. 训练 Baseline
model_win, hist_win = train_experiment(train_loader_win, test_loader_win, "Baseline (Window)", epochs=30)
torch.save(model_win.state_dict(), "model_window.pth")

# B. 训练 ESM
model_esm, hist_esm = train_experiment(train_loader_esm, test_loader_esm, "Advanced (ESM)", epochs=30)
torch.save(model_esm.state_dict(), "model_esm.pth")

# ==========================================
# 7. 结果评估与可视化 (Loss + MSE/R2)
# ==========================================

# 🔥 新增函数：获取所有预测值以计算 R2/MSE
def get_all_preds(model, loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch in loader:
            batch = batch.to(device)
            out = model(batch)
            all_preds.append(out.cpu().numpy())
            all_labels.append(batch.y.cpu().numpy())
    return np.concatenate(all_preds), np.concatenate(all_labels)

# 获取预测结果
preds_win, y_win = get_all_preds(model_win, test_loader_win)
preds_esm, y_esm = get_all_preds(model_esm, test_loader_esm)

# 计算指标
mse_win = mean_squared_error(y_win, preds_win)
r2_win = r2_score(y_win, preds_win)
mse_esm = mean_squared_error(y_esm, preds_esm)
r2_esm = r2_score(y_esm, preds_esm)

# --- 绘图 (Loss + Scatter with Metrics) ---
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Subplot 1: Test Loss Curve
axes[0].plot(hist_win['test_loss'], '--', color='gray', label='Baseline (Window)', linewidth=2)
axes[0].plot(hist_esm['test_loss'], '-', color='#1f77b4', label='Advanced (ESM)', linewidth=2)
axes[0].set_title("Test Loss Curve (Generalization)", fontsize=12, fontweight='bold')
axes[0].set_xlabel("Epoch")
axes[0].set_ylabel("MSE Loss")
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Subplot 2: Predicted vs Actual Scatter
axes[1].scatter(y_win, preds_win, color='gray', alpha=0.5, label='Baseline', s=30)
axes[1].scatter(y_esm, preds_esm, color='#1f77b4', alpha=0.5, label='ESM', s=30)
axes[1].plot([0, 1], [0, 1], 'r--', label='Perfect Fit') # 对角线

# 🔥 显示 MSE 和 R2
text_win = f"Baseline:\nMSE = {mse_win:.4f}\n$R^2$ = {r2_win:.4f}"
text_esm = f"ESM:\nMSE = {mse_esm:.4f}\n$R^2$ = {r2_esm:.4f}"

# 在图中添加文本框
props = dict(boxstyle='round', facecolor='white', alpha=0.8)
axes[1].text(0.05, 0.95, text_win, transform=axes[1].transAxes, fontsize=10,
             verticalalignment='top', bbox=props, color='gray', fontweight='bold')
axes[1].text(0.05, 0.75, text_esm, transform=axes[1].transAxes, fontsize=10,
             verticalalignment='top', bbox=props, color='#1f77b4', fontweight='bold')

axes[1].set_title(f"Predicted vs Actual Solubility\n(Metric Evaluation)", fontsize=12, fontweight='bold')
axes[1].set_xlabel("True Solubility")
axes[1].set_ylabel("Predicted Solubility")
axes[1].legend(loc='lower right')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# ==========================================
# 8. 可解释性分析 (Heatmap + Stacked Bars)
# ==========================================

def compute_saliency(model, data):
    model.eval()
    data = data.to(device)
    data.x.requires_grad_() # 开启梯度
    out = model(data)
    out.backward()
    grads = data.x.grad
    # L2 Norm
    saliency = grads.pow(2).sum(dim=1).sqrt()
    # Normalize 0-1
    saliency = (saliency - saliency.min()) / (saliency.max() - saliency.min() + 1e-8)
    return saliency.cpu().detach().numpy(), out.item()

# 选取测试集样本
test_idx = 0
sample_win = test_dataset_win[test_idx]
sample_esm = test_dataset_esm[test_idx]

# 计算
model_win.load_state_dict(torch.load("model_window.pth"))
model_esm.load_state_dict(torch.load("model_esm.pth"))
s_win, p_win = compute_saliency(model_win, sample_win)
s_esm, p_esm = compute_saliency(model_esm, sample_esm)

# 截取前 50 个残基绘图 (为了清晰)
viz_len = min(len(s_win), 50)
residues = list(range(1, viz_len+1))
s_win = s_win[:viz_len]
s_esm = s_esm[:viz_len]

# 🔥 绘图：热力图 + 拼在一起的柱状图 (Win在上, ESM在下)
fig, axes = plt.subplots(3, 1, figsize=(14, 12), sharex=True, gridspec_kw={'height_ratios': [1, 2, 2], 'hspace': 0.1})

# A. 热力图
sns.heatmap(np.vstack([s_win, s_esm]), cmap="Reds", ax=axes[0], cbar_kws={'label': 'Imp.'},
            yticklabels=["Baseline", "ESM"], xticklabels=residues)
axes[0].set_title(f"A. Saliency Heatmap (Predictions: Win={p_win:.2f}, ESM={p_esm:.2f})", fontweight='bold')
axes[0].set_yticks([0.5, 1.5])

# B. Baseline Bar (Top)
axes[1].bar(residues, s_win, color='gray', alpha=0.6, label='Baseline (Local)')
axes[1].set_ylabel("Importance Score")
# 移除 title，放在图例或文字说明里更简洁，或者使用这种方式：
axes[1].text(0.01, 0.9, "B. Baseline Model (Window Graph)", transform=axes[1].transAxes, fontweight='bold')
axes[1].grid(axis='y', linestyle='--', alpha=0.5)
axes[1].legend(loc='upper right')

# C. ESM Bar (Bottom) - 拼在 Win 下面
colors = ['red' if s > 0.6 else '#1f77b4' for s in s_esm]
axes[2].bar(residues, s_esm, color=colors, alpha=0.8, label='ESM (Structure)')
axes[2].set_ylabel("Importance Score")
axes[2].set_xlabel("Residue Position")
axes[2].text(0.01, 0.9, "C. ESM Model (Structure Graph)", transform=axes[2].transAxes, fontweight='bold')
axes[2].grid(axis='y', linestyle='--', alpha=0.5)
axes[2].legend(loc='upper right')
# 添加高亮说明
axes[2].text(0.02, 0.8, "Red = High Importance (>0.6)", transform=axes[2].transAxes,
             fontsize=9, bbox=dict(facecolor='white', alpha=0.8))

plt.suptitle("Interpretability Analysis: Gradient Saliency Comparison", fontsize=16, y=0.96)
plt.tight_layout() # 可能会报 warning 但不影响
plt.subplots_adjust(top=0.92) # 留出 title 空间
plt.show()

# **第2部分：接下来试试Transformer**

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import seaborn as sns
import math
import os
from sklearn.metrics import r2_score, mean_squared_error # 🔥 新增：用于计算R2和MSE

# ==========================================
# 1. 配置与数据加载
# ==========================================
# 设置随机种子
torch.manual_seed(42)
np.random.seed(42)
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.dpi'] = 120
plt.rcParams['font.sans-serif'] = ['Arial'] # 防止中文乱码
plt.rcParams['axes.unicode_minus'] = False

# 设备配置
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# 读取数据函数
def load_data():
    if os.path.exists('eSol_train.csv') and os.path.exists('eSol_test.csv'):
        print("📦 正在加载 CSV 文件...")
        train_df = pd.read_csv('eSol_train.csv')
        test_df = pd.read_csv('eSol_test.csv')
    else:
        print("⚠️ 未找到数据集文件，生成伪数据用于演示...")
        # 生成随机长度的序列
        def gen_seq():
            length = np.random.randint(50, 500)
            return ''.join(np.random.choice(list("ACDEFGHIKLMNPQRSTVWY"), length))

        dummy_train = {'sequence': [gen_seq() for _ in range(100)], 'solubility': np.random.rand(100)}
        dummy_test = {'sequence': [gen_seq() for _ in range(20)], 'solubility': np.random.rand(20)}
        train_df = pd.DataFrame(dummy_train)
        test_df = pd.DataFrame(dummy_test)
    return train_df, test_df

# 加载数据
train_df, test_df = load_data()

# 🔥【关键步骤】动态计算最大序列长度
# 获取所有序列
all_sequences = train_df['sequence'].tolist() + test_df['sequence'].tolist()
# 找到最长序列的长度
max_seq_len_in_data = max([len(seq) for seq in all_sequences])
# 加上特殊 token (<cls>, <eos>) 的预留空间
MAX_LEN = max_seq_len_in_data + 2

print(f"📊 数据集统计:")
print(f"   - 训练样本数: {len(train_df)}")
print(f"   - 测试样本数: {len(test_df)}")
print(f"   - 数据中最长序列: {max_seq_len_in_data}")
print(f"   - 设定模型 MAX_LEN: {MAX_LEN} (含特殊token)")

# ==========================================
# 2. Tokenizer 与 Dataset
# ==========================================
AA_VOCAB = "<pad> <cls> <eos> <unk> A C D E F G H I K L M N P Q R S T V W Y".split()
aa_to_id = {aa: i for i, aa in enumerate(AA_VOCAB)}
id_to_aa = {i: aa for aa, i in aa_to_id.items()}

class ProteinTokenizer:
    def __init__(self, max_len):
        self.max_len = max_len

    def encode(self, seq):
        # 截断 (以防万一有新数据超过统计长度)
        seq = seq[:self.max_len - 2]

        ids = [aa_to_id.get(aa, aa_to_id['<unk>']) for aa in seq]
        ids = [aa_to_id['<cls>']] + ids + [aa_to_id['<eos>']]

        # Padding
        n_pads = self.max_len - len(ids)
        mask = [False] * len(ids) + [True] * n_pads # True 表示是 Padding，需要被 Mask
        ids = ids + [aa_to_id['<pad>']] * n_pads

        return torch.tensor(ids, dtype=torch.long), torch.tensor(mask, dtype=torch.bool)

class ProteinSequenceDataset(Dataset):
    def __init__(self, df, tokenizer):
        self.seqs = df['sequence'].tolist()
        self.labels = pd.to_numeric(df['solubility'], errors='coerce').fillna(0.0).tolist()
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.seqs)

    def __getitem__(self, idx):
        seq = self.seqs[idx]
        label = self.labels[idx]
        input_ids, padding_mask = self.tokenizer.encode(seq)
        return {
            'input_ids': input_ids,
            'padding_mask': padding_mask,
            'labels': torch.tensor(label, dtype=torch.float)
        }

# 初始化 Tokenizer 和 Dataset
tokenizer = ProteinTokenizer(max_len=MAX_LEN)
train_ds = ProteinSequenceDataset(train_df, tokenizer)
test_ds = ProteinSequenceDataset(test_df, tokenizer)

# DataLoader
BATCH_SIZE = 16 # 如果显存不够，请调小此值
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)

# ==========================================
# 3. Transformer 模型搭建
# ==========================================
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        # 创建足够大的位置编码矩阵
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        # x: [Batch, Seq_Len, Dim]
        # 动态截取对应长度的位置编码
        return x + self.pe[:, :x.size(1), :]

class ProteinTransformer(nn.Module):
    def __init__(self, vocab_size, max_len, d_model=128, nhead=4, num_layers=2):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        # 传入 max_len 确保 PE 矩阵足够大
        self.pos_encoder = PositionalEncoding(d_model, max_len=max_len)

        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead,
                                                   dim_feedforward=256,
                                                   batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.fc_out = nn.Linear(d_model, 1)
        self.dropout = nn.Dropout(0.1)

    def forward(self, src, src_key_padding_mask=None, return_embedding=False):
        # 1. Embedding
        x = self.embedding(src)

        # Hook for Interpretability
        if return_embedding:
            return x

        x = x * math.sqrt(self.embedding.embedding_dim)
        x = self.pos_encoder(x)
        x = self.dropout(x)

        # 2. Transformer
        output = self.transformer_encoder(x, src_key_padding_mask=src_key_padding_mask)

        # 3. Global Pooling (忽略 Padding)
        if src_key_padding_mask is not None:
            mask = (~src_key_padding_mask).unsqueeze(-1).float() # [B, L, 1]
            output = output * mask
            pooled = output.sum(dim=1) / (mask.sum(dim=1) + 1e-9)
        else:
            pooled = output.mean(dim=1)

        # 4. Regression
        return self.fc_out(pooled)

    def forward_with_embeddings(self, embeddings, src_key_padding_mask=None):
        """用于基于梯度的可解释性分析"""
        x = embeddings * math.sqrt(self.embedding.embedding_dim)
        x = self.pos_encoder(x)
        output = self.transformer_encoder(x, src_key_padding_mask=src_key_padding_mask)

        if src_key_padding_mask is not None:
            mask = (~src_key_padding_mask).unsqueeze(-1).float()
            output = output * mask
            pooled = output.sum(dim=1) / (mask.sum(dim=1) + 1e-9)
        else:
            pooled = output.mean(dim=1)

        return self.fc_out(pooled)

# 初始化模型
model = ProteinTransformer(vocab_size=len(AA_VOCAB), max_len=MAX_LEN).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

# ==========================================
# 4. 训练与评估
# ==========================================
def train_step(model, loader):
    model.train()
    total_loss = 0
    for batch in loader:
        ids = batch['input_ids'].to(device)
        mask = batch['padding_mask'].to(device)
        labels = batch['labels'].to(device).unsqueeze(1)

        optimizer.zero_grad()
        outputs = model(ids, src_key_padding_mask=mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

def eval_step(model, loader):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in loader:
            ids = batch['input_ids'].to(device)
            mask = batch['padding_mask'].to(device)
            labels = batch['labels'].to(device).unsqueeze(1)

            outputs = model(ids, src_key_padding_mask=mask)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
    return total_loss / len(loader)

print("🚀 开始训练...")
epochs = 30
history = {'train': [], 'test': []}

for epoch in range(epochs):
    train_loss = train_step(model, train_loader)
    test_loss = eval_step(model, test_loader)

    history['train'].append(train_loss)
    history['test'].append(test_loss)

    if (epoch+1) % 5 == 0:
        print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.4f} | Test Loss: {test_loss:.4f}")

# 🔥 获取所有预测结果以计算 R2 和 MSE
def get_all_preds(model, loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch in loader:
            ids = batch['input_ids'].to(device)
            mask = batch['padding_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(ids, src_key_padding_mask=mask)
            all_preds.append(outputs.cpu().numpy().flatten())
            all_labels.append(labels.cpu().numpy().flatten())

    return np.concatenate(all_preds), np.concatenate(all_labels)

# 计算指标
y_pred, y_true = get_all_preds(model, test_loader)
mse = mean_squared_error(y_true, y_pred)
r2 = r2_score(y_true, y_pred)

# --- 绘图：Loss 曲线 + 散点回归图 ---
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# 图1：训练过程 Loss
axes[0].plot(history['train'], label='Train Loss', marker='.')
axes[0].plot(history['test'], label='Test Loss', marker='.')
axes[0].set_title(f"Transformer Training Loss (MaxLen={MAX_LEN})", fontweight='bold')
axes[0].set_xlabel("Epoch")
axes[0].set_ylabel("MSE Loss")
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 图2：真实值 vs 预测值 (散点图)
axes[1].scatter(y_true, y_pred, alpha=0.6, color='#1f77b4', edgecolor='k', s=40)
axes[1].plot([0, 1], [0, 1], 'r--', label='Perfect Fit') # 对角线
axes[1].set_title(f"Predicted vs True Solubility\n(Test Set Evaluation)", fontweight='bold')
axes[1].set_xlabel("True Solubility")
axes[1].set_ylabel("Predicted Solubility")

# 🔥 在图中显示指标
text_str = f"MSE = {mse:.4f}\n$R^2$ = {r2:.4f}"
props = dict(boxstyle='round', facecolor='white', alpha=0.8)
axes[1].text(0.05, 0.95, text_str, transform=axes[1].transAxes, fontsize=12,
             verticalalignment='top', bbox=props, fontweight='bold', color='darkred')

axes[1].legend(loc='lower right')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# ==========================================
# 5. 可解释性分析 (Saliency Map)
# ==========================================
print("🔍 生成可解释性分析图...")

def compute_saliency(model, idx, dataset):
    model.eval()
    sample = dataset[idx]
    ids = sample['input_ids'].unsqueeze(0).to(device)
    mask = sample['padding_mask'].unsqueeze(0).to(device)

    # 1. Get Embeddings & Require Grad
    embeddings = model(ids, src_key_padding_mask=mask, return_embedding=True)
    embeddings.retain_grad()

    # 2. Forward
    output = model.forward_with_embeddings(embeddings, src_key_padding_mask=mask)

    # 3. Backward
    model.zero_grad()
    output.backward()

    # 4. Compute Saliency
    grads = embeddings.grad[0] # [Seq, Dim]
    saliency = grads.pow(2).sum(dim=1).sqrt().cpu().numpy() # L2 Norm

    # Normalize
    saliency = (saliency - saliency.min()) / (saliency.max() - saliency.min() + 1e-9)
    return saliency, sample['input_ids'], output.item()

# 选取一个测试样本
idx = 0
saliency, token_ids, pred = compute_saliency(model, idx, test_ds)

# 还原序列 (去除 Pad)
tokens = token_ids.numpy()
real_len = 0
for i, t in enumerate(tokens):
    if t == aa_to_id['<pad>']: # 找到第一个 pad
        break
    real_len = i + 1

# 截取有效部分
saliency = saliency[:real_len]
seq_chars = [id_to_aa[t] for t in tokens[:real_len]]

# 限制绘图长度 (如果序列特别长，只画前60个以便展示)
viz_limit = min(real_len, 60)
saliency_viz = saliency[:viz_limit]
chars_viz = seq_chars[:viz_limit]

# --- 绘图 ---
fig, axes = plt.subplots(2, 1, figsize=(15, 8), gridspec_kw={'height_ratios': [1, 2]})

# 1. 热力图
sns.heatmap([saliency_viz], cmap="Reds", ax=axes[0], cbar_kws={'label': 'Gradient Imp.'},
            xticklabels=chars_viz, yticklabels=['Seq'])
axes[0].set_title(f"A. Saliency Heatmap (Pred: {pred:.2f})", fontweight='bold')

# 2. 柱状图
colors = ['red' if s > 0.5 else '#1f77b4' for s in saliency_viz]
axes[1].bar(range(len(saliency_viz)), saliency_viz, color=colors, alpha=0.8)
axes[1].set_xticks(range(len(saliency_viz)))
axes[1].set_xticklabels(chars_viz, fontsize=9)
axes[1].set_title("B. Residue Importance Profile", fontweight='bold')
axes[1].set_ylabel("Importance Score")
axes[1].set_xlabel("Sequence Position")

# 高亮标注
axes[1].text(0.01, 0.9, "Red = High Impact (>0.5)", transform=axes[1].transAxes,
             bbox=dict(facecolor='white', alpha=0.8))

plt.tight_layout()
plt.show()
print("✅ 完成。")

# **第3部分：很好，最后试试多模态**

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch_geometric.data import Data, Batch
from torch_geometric.nn import GCNConv, global_mean_pool
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, r2_score
import os
import math
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel

# ==========================================
# 1. 全局配置 & ESM 模型加载
# ==========================================
torch.manual_seed(42)
np.random.seed(42)
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.dpi'] = 120
plt.rcParams['font.sans-serif'] = ['Arial']
plt.rcParams['axes.unicode_minus'] = False

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# 加载 ESM-2 模型 (用于提取 ESM 特征分支的输入)
ESM_MODEL_NAME = "facebook/esm2_t6_8M_UR50D"
print(f"⏳ Loading ESM Model: {ESM_MODEL_NAME}...")
try:
    esm_tokenizer = AutoTokenizer.from_pretrained(ESM_MODEL_NAME)
    esm_model = AutoModel.from_pretrained(ESM_MODEL_NAME, attn_implementation="eager").eval().to(device)
    print("✅ ESM Model loaded!")
except Exception as e:
    print(f"⚠️ ESM Load Error: {e}")
    exit()

# ==========================================
# 2. 特征提取工具函数
# ==========================================

# --- A. GNN 工具 (One-Hot + Window) ---
AA_VOCAB = "ACDEFGHIKLMNPQRSTVWY"
aa_to_int = {aa: i for i, aa in enumerate(AA_VOCAB)}

def get_one_hot(seq):
    L = len(seq)
    x = torch.zeros(L, 20)
    for i, aa in enumerate(seq):
        if aa in aa_to_int:
            x[i, aa_to_int[aa]] = 1.0
    return x

def get_window_adj(seq, window=2):
    L = len(seq)
    src, dst = [], []
    for i in range(L):
        # Self loop
        src.append(i); dst.append(i)
        start = max(0, i - window)
        end = min(L, i + window + 1)
        for j in range(start, end):
            if i != j:
                src.append(i); dst.append(j)
    return torch.tensor([src, dst], dtype=torch.long)

# --- B. Transformer 工具 (Tokenizer) ---
TRANS_VOCAB = "<pad> <cls> <eos> <unk> A C D E F G H I K L M N P Q R S T V W Y".split()
trans_to_id = {aa: i for i, aa in enumerate(TRANS_VOCAB)}

class SimpleTokenizer:
    def __init__(self, max_len):
        self.max_len = max_len

    def encode(self, seq):
        seq = seq[:self.max_len - 2]
        ids = [trans_to_id.get(aa, trans_to_id['<unk>']) for aa in seq]
        ids = [trans_to_id['<cls>']] + ids + [trans_to_id['<eos>']]
        n_pads = self.max_len - len(ids)
        mask = [False] * len(ids) + [True] * n_pads
        ids = ids + [trans_to_id['<pad>']] * n_pads
        return torch.tensor(ids, dtype=torch.long), torch.tensor(mask, dtype=torch.bool)

# --- C. ESM 工具 (Pre-trained Embedding) ---
def get_esm_embedding(seq):
    """提取整条序列的 ESM 特征向量 (320维)"""
    seq = seq[:1020]
    inputs = esm_tokenizer(seq, return_tensors="pt", truncation=True, max_length=1022).to(device)
    with torch.no_grad():
        outputs = esm_model(**inputs)
    # 取 [CLS] token 或者 Mean Pooling
    # 这里使用 Mean Pooling (去除 cls/eos)
    embedding = outputs.last_hidden_state[0, 1:-1, :].mean(dim=0).cpu()
    return embedding # [320]

# ==========================================
# 3. 综合数据集 (All-in-One Dataset)
# ==========================================

class MultiModalDataset(Dataset):
    def __init__(self, df, tokenizer):
        self.data = []
        print(f"📦 Processing {len(df)} sequences...")

        for _, row in tqdm(df.iterrows(), total=len(df), desc="Building Data"):
            try:
                seq = row['sequence']
                label = float(row['solubility'])

                # 1. GNN Data
                gnn_x = get_one_hot(seq)
                gnn_edge_index = get_window_adj(seq)

                # 2. Transformer Data
                trans_ids, trans_mask = tokenizer.encode(seq)

                # 3. ESM Data
                esm_emb = get_esm_embedding(seq)

                self.data.append({
                    'gnn_x': gnn_x,
                    'gnn_edge_index': gnn_edge_index,
                    'trans_ids': trans_ids,
                    'trans_mask': trans_mask,
                    'esm_emb': esm_emb,
                    'y': torch.tensor([label], dtype=torch.float)
                })
            except:
                continue

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

def collate_fn(batch):
    # GNN Batch
    gnn_list = [Data(x=i['gnn_x'], edge_index=i['gnn_edge_index']) for i in batch]
    gnn_batch = Batch.from_data_list(gnn_list)

    # Trans Batch
    trans_ids = torch.stack([i['trans_ids'] for i in batch])
    trans_mask = torch.stack([i['trans_mask'] for i in batch])

    # ESM Batch
    esm_emb = torch.stack([i['esm_emb'] for i in batch])

    # Label
    y = torch.stack([i['y'] for i in batch])

    return {
        'gnn_batch': gnn_batch,
        'trans_ids': trans_ids,
        'trans_mask': trans_mask,
        'esm_emb': esm_emb,
        'y': y
    }

# --- Load Data ---
if os.path.exists('eSol_train.csv'):
    train_df = pd.read_csv('eSol_train.csv')#.iloc[:50] # Debug
    test_df = pd.read_csv('eSol_test.csv')#.iloc[:20]
else:
    print("⚠️ Using Dummy Data")
    dummy = {'sequence': ['MKTVRQERLKSIVRILERSKEPVSGAQLAEELSVSRQVIVQDIAYLRSLGYNIVATPRGYVLAGG']*20, 'solubility': np.random.rand(20)}
    train_df = pd.DataFrame(dummy)
    test_df = pd.DataFrame(dummy)

# Dynamic Max Len
all_seqs = train_df['sequence'].tolist() + test_df['sequence'].tolist()
MAX_LEN = max([len(s) for s in all_seqs]) + 2
print(f"📊 Max Len: {MAX_LEN}")

tokenizer = SimpleTokenizer(MAX_LEN)
train_ds = MultiModalDataset(train_df, tokenizer)
test_ds = MultiModalDataset(test_df, tokenizer)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_ds, batch_size=32, shuffle=False, collate_fn=collate_fn)

# ==========================================
# 4. 模块化模型定义
# ==========================================

# --- Module 1: GNN (Window) ---
class GNNBranch(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(20, 64)
        self.conv2 = GCNConv(64, 64)
        self.conv3 = GCNConv(64, 32)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = F.relu(self.conv3(x, edge_index))
        x = global_mean_pool(x, batch) # [B, 32]
        return x

# --- Module 2: Transformer (Seq) ---
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe.unsqueeze(0))
    def forward(self, x): return x + self.pe[:, :x.size(1), :]

class TransBranch(nn.Module):
    def __init__(self, vocab_size, max_len):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, 64)
        self.pos_encoder = PositionalEncoding(64, max_len)
        layer = nn.TransformerEncoderLayer(d_model=64, nhead=4, dim_feedforward=128, batch_first=True)
        self.transformer = nn.TransformerEncoder(layer, num_layers=2)
        self.fc = nn.Linear(64, 32)

    def forward(self, src, mask):
        x = self.embedding(src) * math.sqrt(64)
        x = self.pos_encoder(x)
        x = self.transformer(x, src_key_padding_mask=mask)
        mask_expanded = (~mask).unsqueeze(-1).float()
        x = (x * mask_expanded).sum(dim=1) / (mask_expanded.sum(dim=1) + 1e-9)
        return self.fc(x) # [B, 32]

# --- Module 3: ESM (Pre-trained) ---
class ESMBranch(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Linear(320, 32) # Project 320 -> 32

    def forward(self, x):
        return F.relu(self.fc(x)) # [B, 32]

# --- Main Wrapper ---
class UnifiedModel(nn.Module):
    def __init__(self, use_gnn=False, use_trans=False, use_esm=False):
        super().__init__()
        self.use_gnn = use_gnn
        self.use_trans = use_trans
        self.use_esm = use_esm

        if use_gnn: self.gnn = GNNBranch()
        if use_trans: self.trans = TransBranch(len(TRANS_VOCAB), MAX_LEN)
        if use_esm: self.esm = ESMBranch()

        # Calculate Fusion Dim
        dim = 0
        if use_gnn: dim += 32
        if use_trans: dim += 32
        if use_esm: dim += 32

        self.regressor = nn.Sequential(
            nn.Linear(dim, 32),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(32, 1)
        )

    def forward(self, batch):
        feats = []
        if self.use_gnn:
            feats.append(self.gnn(batch['gnn_batch']))
        if self.use_trans:
            feats.append(self.trans(batch['trans_ids'], batch['trans_mask']))
        if self.use_esm:
            feats.append(self.esm(batch['esm_emb']))

        combined = torch.cat(feats, dim=1)
        return self.regressor(combined)

# ==========================================
# 5. 训练循环 (Epochs=50)
# ==========================================

def run_experiment(name, use_gnn, use_trans, use_esm, epochs=50):
    print(f"\n🚀 Experiment: {name}")
    model = UnifiedModel(use_gnn, use_trans, use_esm).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.MSELoss()

    loss_history = []

    # Progress Bar
    pbar = tqdm(range(epochs), desc=f"Training {name}", unit="ep")
    for epoch in pbar:
        model.train()
        ep_loss = 0
        for batch in train_loader:
            # Move to device
            batch_data = {
                'gnn_batch': batch['gnn_batch'].to(device),
                'trans_ids': batch['trans_ids'].to(device),
                'trans_mask': batch['trans_mask'].to(device),
                'esm_emb': batch['esm_emb'].to(device)
            }
            y = batch['y'].to(device)

            optimizer.zero_grad()
            out = model(batch_data)
            loss = criterion(out, y)
            loss.backward()
            optimizer.step()
            ep_loss += loss.item()

        avg_loss = ep_loss / len(train_loader)
        loss_history.append(avg_loss)
        pbar.set_postfix({'Loss': f"{avg_loss:.4f}"})

    # Evaluation
    model.eval()
    preds, labels = [], []
    with torch.no_grad():
        for batch in test_loader:
            batch_data = {
                'gnn_batch': batch['gnn_batch'].to(device),
                'trans_ids': batch['trans_ids'].to(device),
                'trans_mask': batch['trans_mask'].to(device),
                'esm_emb': batch['esm_emb'].to(device)
            }
            y = batch['y'].to(device)
            out = model(batch_data)
            preds.append(out.cpu().numpy())
            labels.append(y.cpu().numpy())

    preds = np.concatenate(preds)
    labels = np.concatenate(labels)
    mse = mean_squared_error(labels, preds)
    r2 = r2_score(labels, preds)

    print(f"   ✅ Result: MSE={mse:.4f}, R2={r2:.4f}")
    return loss_history, mse, r2, preds, labels

# --- Run 4 Experiments ---
# 1. GNN Only
h1, m1, r1, p1, y1 = run_experiment("GNN Only", True, False, False)
# 2. Transformer Only
h2, m2, r2, p2, y2 = run_experiment("Trans Only", False, True, False)
# 3. ESM Only (Linear Probe)
h3, m3, r3, p3, y3 = run_experiment("ESM Only", False, False, True)
# 4. Fusion (GNN+Trans+ESM)
h4, m4, r4, p4, y4 = run_experiment("Fusion (All)", True, True, True)

# ==========================================
# 6. 可视化对比
# ==========================================
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# 1. Loss Curves
axes[0].plot(h1, label='GNN', linestyle=':')
axes[0].plot(h2, label='Trans', linestyle='--')
axes[0].plot(h3, label='ESM', linestyle='-.')
axes[0].plot(h4, label='Fusion', linewidth=2, color='red')
axes[0].set_title("Training Loss")
axes[0].legend()

# 2. Metrics (MSE & R2)
models = ['GNN', 'Trans', 'ESM', 'Fusion']
mse_list = [m1, m2, m3, m4]
r2_list = [r1, r2, r3, r4]
x = np.arange(4); w = 0.35

ax2 = axes[1]
ax2.bar(x - w/2, mse_list, w, label='MSE (Lower better)', color='salmon', alpha=0.8)
ax2.bar(x + w/2, r2_list, w, label='R2 (Higher better)', color='skyblue', alpha=0.8)
ax2.set_xticks(x); ax2.set_xticklabels(models)
ax2.set_title("Model Comparison")
ax2.legend()
# Labels
for i, v in enumerate(mse_list): ax2.text(i-w/2, v, f"{v:.3f}", ha='center', va='bottom', fontsize=8)
for i, v in enumerate(r2_list): ax2.text(i+w/2, v, f"{v:.3f}", ha='center', va='bottom', fontsize=8)

# 3. Scatter (Best Model: Fusion)
ax3 = axes[2]
ax3.scatter(y4, p4, alpha=0.6, c='purple', s=40)
ax3.plot([0,1], [0,1], 'r--')
ax3.set_title(f"Fusion Model Pred (R2={r4:.3f})")
ax3.set_xlabel("True Solubility")
ax3.set_ylabel("Predicted Solubility")

plt.tight_layout()
plt.show()